In [1]:
import cv2 as cv
import numpy as np
import random
import os
import math

In [2]:
def click_corners(img, objpoints, imgpoints, objp, window, n_rows=9, n_cols=6):
    """
    get 4 corners from user clicks (in order) and use linear interpolation to get the
    other inner points.
    """

    order = ["TL", "BL", "TR", "BR"]
    box_corners = []

    # function to handle clicks on the image
    def click_event(event, x, y, flags, param):

        box_corners, img, window = param

        # check for left mouse clicks
        if event == cv.EVENT_LBUTTONDOWN and len(box_corners) < 4:

            box_corners.append((x, y))

            # display clicked points
            # on the image window
            font = cv.FONT_HERSHEY_SIMPLEX
            cv.putText(
                img, order[len(box_corners) - 1], (x, y), font, 1, (255, 0, 0), 2
            )
            cv.imshow(window, img)

    cv.namedWindow(window, cv.WINDOW_NORMAL)
    cv.imshow(window, img)
    cv.setMouseCallback(window, click_event, [box_corners, img, window])
    # wait for a key to be pressed to exit
    print("click on the 4 corners, then press any key.")
    cv.waitKey(0)

    tl, bl, tr, br = np.array(box_corners, dtype=np.float32)

    # get first and last columns of points with linear interpolation
    first_col = np.linspace(tl, bl, n_rows)
    last_col = np.linspace(tr, br, n_rows)

    # get rest corner points by linearly interpolating the two columns
    all_points = np.vstack(
        [np.linspace(first_col[i], last_col[i], n_cols) for i in range(n_rows)]
    )
    corners = all_points.reshape(-1, 1, 2)

    objpoints.append(objp)
    imgpoints.append(corners)

    cv.destroyWindow(window)
    cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
    cv.imshow("Interpolated Chessboard Corners", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
n_rows = 8
n_cols = 6



objp = np.zeros((n_rows * n_cols, 3), np.float32)


objp[:, :2] = np.mgrid[0:n_rows, 0:n_cols].T.reshape(-1, 2)


# objp= objp*



# Lists to store object and image points from all images.


objpoints = []  # 3d point in real world space


imgpoints = []  # 2d points in image plane.



for i in range(1, 5):


    path = f"./Voxel-based 3D Reconstruction/data/cam{i}/intrinsics.avi"
    cam = cv.VideoCapture(path)


    j = 0


    while cam.isOpened():


        ret, frame = cam.read()
        j += 1
        # Write out every 15th frame until we reach 300 frames
        if j % 15 == 0:
            out_path = os.path.join("./data/samples", f"image{i}_{j}.jpg")
            cv.imwrite(out_path, frame)
            print(f"Saved {out_path}")

        if j >= 300:
            # We only wanted up to 300 frames
            break

    cam.release()

    # click_corners(
    #     img,
    #     objpoints,
    #     imgpoints,
    #     objp,
    #     window=window,
    #     n_rows=n_rows,
    #     n_cols=n_cols,
    # )

    # for frame in os.listdir("./data"):
    #     if not frame.lower().endswith((".jpg", ".png", ".jpeg")):
    #         continue


    #     img = cv.imread(frame)
    #     if img is None:
    #         continue
    #     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    #     window = f"image_{frame}"
    #     click_corners(
    #         img,
    #         objpoints,
    #         imgpoints,
    #         objp,
    #         window=window,
    #         n_rows=n_rows,
    #         n_cols=n_cols,
    #     )

    # ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(


    #     objpoints, imgpoints, img.shape[::-1], None, None


    # )